In [ ]:
#| default_exp utils

In [ ]:
#| export
import string
import uuid
import functools
import asyncio

In [ ]:
#| export
def create_nano_id(size=12):
    # Define characters to use (alphanumeric)
    alphabet = string.ascii_letters + string.digits
    
    # Generate UUID and convert to int
    uuid_int = uuid.uuid4().int
    
    # Convert to base62
    result = ""
    while uuid_int:
        uuid_int, remainder = divmod(uuid_int, len(alphabet))
        result = alphabet[remainder] + result
    
    # Pad if necessary and return desired length
    return result[:size]

In [ ]:
# | export
def async_to_sync(async_func):
    """Convert an async function to a sync function"""
    @functools.wraps(async_func)
    def sync_wrapper(*args, **kwargs):
        try:
            loop = asyncio.get_event_loop()
            if loop.is_running():
                import concurrent.futures
                with concurrent.futures.ThreadPoolExecutor() as executor:
                    future = executor.submit(asyncio.run, async_func(*args, **kwargs))
                    return future.result()
            else:
                return loop.run_until_complete(async_func(*args, **kwargs))
        except RuntimeError:
            return asyncio.run(async_func(*args, **kwargs))
    return sync_wrapper